## Tutorial for utilizing pCCA-FA for neural population activity

In [ ]:
# Imports and params
import numpy as np
import pcca_fa_mdl as pf
import sim_pcca_fa as spf
from timeit import default_timer as timer
import matplotlib.pyplot as plt

# random seed for reproducibility
rand_seed = 19

# plot colors
color_map = {
    'across':np.array([255,76,178])/255, # across-area
    'within1':np.array([111,192,255])/255, # within-area, area 1
    'within2':np.array([0,87,154])/255, # within-area, area 2
    'within':np.array([0,144,255])/255, # within-area, combined
}


First, we need data that will be fed into the pCCA-FA model. In the following cell, we will simulate data according to the generative model specifed by pCCA-FA to use. But, any two spike count matrices ($X_1 \in \mathbb{R}^{N \times n_1}, X_2 \in \mathbb{R}^{N \times n_2}$) can be used in place of this.

In [ ]:
# parameters for simulating data
n1,n2 = 10,10 # this parameter indicates the number of neurons in area 1 and area 2
d,d1,d2 = 3,2,1 # this parameter indicates the number of latent variables for across-area, within-area 1, and within-area 2
N = 5000 # number of trials or observations

# simulate data according to generative model pCCA-FA
pf_simulator = spf.sim_pcca_fa(n1=n1,n2=n2,d=d,d1=d1,d2=d2,rand_seed=rand_seed)
X_1,X_2 = pf_simulator.sim_data(N,rand_seed=rand_seed)
sim_params = pf_simulator.get_params()

print('Successfully generated spike count matrices X_1 and X_2 with shape {} and {}'.format(X_1.shape, X_2.shape))

Now, the variables X_1 and X_2 represent our two areas' spike count matrices. Each row is an observation (trials). Each column is the simulated activity of a neuron. The next step is to fit the model. We need to decide which latent dimensionalities to test. This will correspond to the dimensionality of latent variables tested during model cross-validation.

In [ ]:
# select dimensionalities to test, needs to be a list of integers
d_list = np.arange(0,6)

Now we get to train the model! First, we initialize it, then fit it by cross-validating the latent dimensionalities for within- and across-area dimensions. Note, this takes a while...

In [ ]:
# train model
model = pf.pcca_fa()
start = timer()
results = model.crossvalidate(X_1,X_2,rand_seed=rand_seed,verbose=True,d_list=d_list,d1_list=d_list,d2_list=d_list,parallelize=True,n_folds=5)
end = timer()
cv_d,cv_d1,cv_d2 = results['d'],results['d1'],results['d2']
print(f'{end-start:.2f} seconds elapsed...')
print(f'Identified dimensionalities - across-area: {cv_d:d}, within-area 1: {cv_d1:d}, within-area 2: {cv_d2:d}')

In place of cross-validating (or after we have completed a round of cross-validation), we can save time by simply fitting a model of given dimensionality using all trials. We can do this below:

In [ ]:
model = pf.pcca_fa()
start = timer()
model.train(X_1,X_2,d=d,d1=d1,d2=d2) # directly specify the dimensionalities
end = timer()
print(f'{end-start:.2f} seconds elapsed...')

Now, we have a model fit. What does it all mean? model_params is a dictionary where each key is a parameter name of the model, and each value is the fit value of that parameter. <br>

- <b>mu</b>: mean firing rate for each neuron
- <b>L_total</b>: combined loadings for across- and within-area latent variables
- <b>W</b>: loadings for across-area variance for each neuron and each across-area latent variable
- <b>L</b>: loadings for within-area variance for each neuron and each within-area latent variable
- <b>psi</b>: independent variance of each neuron
- <b>d</b>: dimensionality for across-area
- <b>d1</b>: dimensionality for within-area 1
- <b>d2</b>: dimensionality for within-area 2

In [ ]:
model_params = model.get_params()
print(model_params.keys())

We can compute data metrics using the model, such as the shared variance explained by the model.

In [ ]:
# compute metrics - from ground truth parameters
sim_model = pf.pcca_fa()
sim_model.set_params(sim_params)
true_psv = sim_model.compute_metrics(cutoff_thresh=0.95)['psv']

# compute estimated percent of shared variance
est_psv = model.compute_psv()

print('True across-area %sv = {:.2f}%, estimated across-area %sv = {:.2f}%'.format(true_psv['avg_psv_W_total'], est_psv['avg_psv_W_total']))

pCCA-FA accurately estimates the %sv in the simulated data. To get a sense of how consistent this is across different datasets, we can generate new simulated data. In this example we will test how pCCA-FA estimates across- and within-area shared variance with different numbers of trials. We will keep the parameters the same, and generate independent datasets for each number of trials.

In [ ]:
n_trials = np.array([100,150,300,600,1000,1500,3000,6000])
n_boots = 30 # repeat 30 times at each trial count

est_psv_across = np.full((n_boots,len(n_trials)),fill_value=np.nan)
est_psv_within = np.full((n_boots,len(n_trials)),fill_value=np.nan)

for i in range(n_boots):
    for j,N in enumerate(n_trials):
        X_1,X_2 = pf_simulator.sim_data(N)

        # fit the data using pcca-fa
        model = pf.pcca_fa()
        model.train(X_1,X_2,d=d,d1=d1,d2=d2)
        est_psv = model.compute_psv()

        # log the estimated %sv
        est_psv_across[i,j] = est_psv['avg_psv_W_total']
        est_psv_within[i,j] = est_psv['avg_psv_L_total']

In the plot below, we can see that with sufficient trials, pCCA-FA accurately estimates the across- and within-area shared variance. When the number of trials is low compared to the number of parameters needed to estimate, small estimation errors ($<5\%$) occurs. See Methods and supplemental information in the preprint for more details.

In [ ]:
# plot formatting
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Arial"
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

fig,ax = plt.subplots(2,1,sharex=True,sharey=True,figsize=(5,6))
fig.tight_layout(pad=2.5)

ax[0].plot([0,np.max(n_trials)],[true_psv['avg_psv_W_total'],true_psv['avg_psv_W_total']],'--', color='gray',label='ground truth') # across
ax[1].plot([0,np.max(n_trials)],[true_psv['avg_psv_L_total'],true_psv['avg_psv_L_total']],'--', color='gray') # within

ax[0].errorbar(n_trials, np.mean(est_psv_across,axis=0), yerr=np.std(est_psv_across,axis=0), fmt='-o', color=color_map['across'], ms=4, capsize=3,label='estimated by pCCA-FA')
ax[1].errorbar(n_trials, np.mean(est_psv_within,axis=0), yerr=np.std(est_psv_within,axis=0), fmt='-o', color=color_map['within'], ms=4, capsize=3)

# plot labels
ax[0].legend()
ax[0].set_xticks(np.arange(0,np.max(n_trials)+1,1000))
ax[0].set_ylabel('estimated across-area %sv',color=color_map['across'])
ax[1].set_ylabel('estimated within-area %sv',color=color_map['within'])
fig.supxlabel('number of trials used to estimate')

plt.show()

The pCCA-FA parameters also yield the canonical correlations, as would be identified by applying traditional CCA to the neural activity. Using the model we trained, we can obtain the canonical directions and canonical correlations as defined in CCA.

In [ ]:
(canonical_dirs_x, canonical_dirs_y), rho = model.get_canonical_directions()

xdata = np.arange(d)+1
fig,ax = plt.subplots()
ax.plot(xdata, rho, marker='o', color='gray')
ax.set_ylim(0,1)
ax.set_ylabel(r'canonical correlation ($\rho$)')
ax.set_xticks(xdata)
ax.set_xlabel('canonical pair number')
plt.show()